In [ ]:
import sys
sys.path.append("../")
from neuropack.devices import BrainFlowDevice
from time import time

# An Introduction to Working with Brainwaves and NeuroPack

NeuroPack allows you to record, analyze, and identify brainwaves gathered using standard EEG devices. By utilizing Brainflow as a backend, NeuroPack can easily communicate with a whole host of different devices. Further, NeuroPack offers you various possibilities for signal filtering and visualization. Moreover, NeuroPack provides a starting point for developing and evaluating feature extraction models for brainwave-based authentication systems.

NeuroPack mainly relies on Brainflow for handling device connections and SciPy for filtering and data processing. Further, it heavily utilizes numpy to make computations as fast as possible. Most components are implemented using abstract base classes, which offer a starting point to extend this library with your ideas.

In this notebook, we provide code examples and explanations demonstrating how to use NeuroPack.

## Connecting and Working with Devices

By utilizing Brainflow, NeuroPack supports several devices out of the box. To allow for even more devices in the future, a simple wrapper is used to define general functions all devices found within the library should support. These functions include obvious stuff like $connect$ and $disconnect$ and more specific functionality like a wear detection, which is handy when implementing an authentication system. In the following code example, a Muse 2 device is created, connected, and its recorded data is printed for 10 seconds. 

Create Muse 2 device using a predefined wrapper function. To connect an arbitrary device supported by Brainflow, please consult the Brainflow documentation for the device's id. Everything else is taken care of in the base "BrainFlowDevice" class.

In [ ]:
muse_device = BrainFlowDevice.CreateMuse2Device()
muse_device.connect()

Explicitly start the data stream, else no data can be fetched from the device. Conceptually, starting the data stream only enables data storage, as even if it is stopped, data should be processed by the device to allow for wear detection. The only reason to turn it off is to keep the memory load low.

In [ ]:
muse_device.start_stream()
start = time()

Fetch data and print it out:

In [ ]:
while time() - start < 10:
    if muse_device.has_data():
        print(muse_device.fetch_data())
muse_device.stop_stream()

## Configure and Use ERP Acquisition Tasks

Within NeuroPack, different tasks to acquire event-related potentials (ERPs) used for authentication can be found. These tasks follow the oddball paradigm, where a specific stimulus is presented infrequently to the user within a series of frequent stimuli. These oddball paradigm tasks have proven themself useful to evoke the P300 ERP, which is, in turn, proven to be unique enough to be used for authentication. 

Examples of all tasks contained within NeuroPack can be found in the "tasks.ipynb" notebook. In the following, we will configure a simple task based on colors. Generally, we have to specify a target stimulus and one or several neutral stimuli for each task. NeuroPack does not take care of instructing which is which, so this needs to be done beforehand. The created task uses blue as the target and green as a non-target stimulus. Further, we specify that between two target stimuli, at least two and at most six non-target stimuli should be presented.

In [ ]:
from neuropack.tasks import ColorTask

# Create a task using blue as target color and green as non-target color. Each stimulus is shown for 300 ms.
target_color = "blue"
non_target_color = "green"
task = ColorTask(2, 6, non_target_color, target_color, 300)

## Collect and Process Data

Once a task and a device have been set up, we can use these to collect the sought-after ERPs needed for brainwave-based authentication. To do this, we need to record data while a task is running. As the persistent storage of data can be complex, NeuroPack provides a simple container that takes care of data storage and handling. This container is called "EEGContainer" and supports the addition of events, e.g., stimuli presentations and raw data. Theoretically, the container can be used to do event management during live recording, yet, in our testing, recording, and processing sequentially proved fast enough.

When creating a container, we need to specify certain parameters. These are the sample rate of the used recording device and the channel names.

In [ ]:
from neuropack import EEGContainer

# Create container with the channel names and sample rate of the device
container = EEGContainer(muse_device.channel_names, muse_device.sample_rate)

Once this is done and the container is created, we can start to record data. To do this, we have to start the stream of the device and the chosen acquisition task. As data from both is stored within a buffer, we can ignore the acquisition data during the recording and take care of it afterward. This is not possible with the EEG data captured by the device, as here, the amount of data is typically many times larger than the buffer size. An example recording fixture might look as follows:

In [ ]:
# Define duration of recording
duration_seconds = 10

# Define timestamp of start
start_time = time()

# Start recording and task
muse_device.start_stream()
task.start()

while time() < start_time + duration_seconds:
    if muse_device.has_data():
        container.add_data(muse_device.fetch_data())

muse_device.stop_stream()

# After EEG recording is done, fetch stimulus timings from task
# This can be done afterwards and before stopping the task
# We can further directly collect the events into a list for further processing
events = []
while task.has_data():
    event = container.add_event(task.fetch_data().timestamp, 100, 500)
    events.append(event)

# Stop task
task.stop()



During recording, we want to ensure that the device used is sitting correctly. This can be done using wear detection. We can modify the first loop in the code above as follows:

In [ ]:
while time() < start_time + duration_seconds and muse_device.is_worn():
    if muse_device.has_data():
        container.add_data(muse_device.fetch_data())

Once data is recorded, we can visualize it both in the time and frequency domain. Data in both EventContainers and EEGContainers can be visualized.

In [ ]:
# Visualize the data in the time domain
container.plot_ch()

# Visualize the data in the frequency domain
container.plot_ps()

Beyond recording, we can also save and load data to and from ".csv" files using the EEGContainer. The added events are marked with an event code during both operations. Per default, NeuroPack uses the event code "1". Yet, especially when working with external data, these codes can change. To handle this, the event code can be specified when loading and saving data using an EEGContainer.

In [ ]:
# Save container to disk
container.save_signals("signals.csv", event_marker="1")

# Create new temporary container from disk
temp = EEGContainer.from_file(muse_device.channel_names, muse_device.sample_rate, "signals.csv")
del temp

Once we have acquired data through recording or loading in a file, we can apply further processing. For this purpose, NeuroPack provides a set of filters and processing functions. Generally, these can be applied to both EEG containers and extracted events. Preprocessing always happens in place. Therefore, it is recommended to save data before applying any processing steps.

In [ ]:
# Applying a detrend filter to whole container
from neuropack.preprocessing import DetrendFilter

DetrendFilter().apply(container)

We often want to apply several filters and preprocessing steps in a given order to our data. To make this easier, NeuroPack includes a preprocessing pipeline that combines several steps and applies them sequentially. Again, the pipeline can be applied to EEG containers and extracted events as EventContainers. In the following example, we want to reduce the channels included in our previously extracted events and apply a bandpass filter. The pipeline can be applied to both a single container and a list of containers.

In [ ]:
# Create pipline with bandpass filter and 
from neuropack.preprocessing import BandpassFilter, ReductionFilter, PreprocessingPipeline

# Create pipeline. The order of the filters is important!
# Filters can be added in the constructor or with the add_filter method
pipeline = PreprocessingPipeline(ReductionFilter("TP9", "TP10"))
pipeline.add_filter(BandpassFilter())

# Apply pipeline to containers
pipeline.apply(events)

## Creating Biometric Templates based on Brainwaves
After collecting and processing data, it is time to create the actual templates used for authentication. This is done using one of the available feature extraction models, which are part of NeuroPack. In this example, we chose the PACModel. In the model, each channel contained in the data is transformed into a set of features:
1) A autoregressive model is fitted to the data. The resulting coefficients are then used as part of the resulting template
2) Several powerbands are extracted from the data and also used as part of the resulting template

The concrete internal workings are not that important for this example. The important part is that the result of all feature extraction models contained in this library are templates in the form of numpy arrays.

In [ ]:
from neuropack import PACModel

# Create a PAC model
model = PACModel()

# Extract features from data
templates = []
for ev in events:
    # Skip event if it contains a blink
    if ev.contains_blink(): continue

    # Extract features from event
    template = model.extract_features(ev)
    templates.append(template)

Now that we have transformed each singular event into a template, we can also convert the average of all events into a template. This is typically done to create a template stored inside the database. We want to minimize noise and create a more robust template by averaging events.

In [ ]:
from neuropack.utils import oavg

average_event = oavg(events)
average_template = model.extract_features(average_event)

Finally, we can compare the resulting templates with one of the included similarity functions. Alternatively, we can also use the templates, which are, as previously stated, just numerical arrays, in any other way we want. In the following, we use the cosine similarity to compare the average template to each singular template.

In [ ]:
from neuropack.similarity_metrics import bounded_cosine_similarity

for t in templates:
    print(bounded_cosine_similarity(average_template, t))